# import def func

In [40]:
# import logging
import sys
# stdout=sys.stdout
sys.path.append('../../')
import os
import random
import time

# import matplotlib
# import matplotlib.pyplot as plt
# import mplfinance as mpf
# sys.stdout=stdout
import numpy as np
import pandas as pd
from JohnsonUtil import commonTips as cct
from JohnsonUtil import johnson_cons as ct
from JSONData import sina_data
from JSONData import tdx_data_Day as tdd
from JSONData import tdx_hdf5_api as h5a

# from mplfinance.original_flavor import candlestick_ohlc
# from JSONData import tdx_hdf5_api as h5a

# %matplotlib widget
# plt.rc('font', family='SimHei', size=13)
# matplotlib.get_backend()
# jupyter nbextension enable --py widgetsnbextension

os.environ['NUMEXPR_MAX_THREADS'] = '12'
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
import nest_asyncio
nest_asyncio.apply()

In [41]:
def show_tdx_data(df, codelist=None, market_value='3', col=None):
    col_src = [
        "name", "couts", "date", "perc", "ra_diff","xratio", "idx", "hat", "close",
        "volsum5d", "hmax", "lmin", "perc1d", "ma20", "ma250", "nclose",
        "llow", 'vol1d', 'vol2d', 'vol3d', 'vol4d', 'vol5d', "perc%sd" %
        (market_value),
        "perc5d"]

    if isinstance(df, pd.Series):
        col_name = [co for co in col_src if co in df.index]
        # col_name.insert(1,'date')
    else:
        col_name = [co for co in col_src if co in df.columns]

    # print(col_name)
    # if isinstance(df, pd.DataFrame) and 'name' in df.columns:
    #     columns_ = col_name
    # else:
    #     if isinstance(df, pd.Series) and 'name' in df.index:
    #         columns_ = col_name.insert(1, "date")
    #     else:
    #         columns_ = col_name.remove("name")
    
    if isinstance(col,list):
        col_name.extend(col)
    elif col is not None:
        col_name.insert(4, col)

    if codelist is None:
        if isinstance(df, pd.Series):
            print("Count:1")
            temp = df.loc[col_name]
        else:
            print("Count:%s %s" % (df.shape))
            temp = df.loc[:, col_name]
    else:
        temp = df.loc[[x for x in codelist if x in df.index], col_name]
    return temp

def clean_sina_dm(dd):
    if (cct.get_now_time_int() > 915 and cct.get_now_time_int() < 926):
        dd = dd[dd.close > 0]
    else:
        dd = dd[dd.open > 0]
    return dd

global sina_dm
sina_dm = None


def get_sina_data_cname(df, dm=None, index=False, realtime=False):
    # index_status=False
    global sina_dm
    if dm is None:
        if sina_dm is None:
            dm = sina_data.Sina().market('all')
        else:
            dm = sina_dm.copy()

    if 'ticktime' in df.columns:
        time_l = time.mktime(dm.loc[:, 'ticktime'][-1].timetuple())
        dur = round((time.time() - time_l) / 60, 2)
        if dur > 5:
            print("timel > 5,renew dm")
            dm = sina_data.Sina().market('all')
            sina_dm = dm.copy()

    if realtime:
        print("compute percent")
        df.rename(columns={'low': 'llow',
                  'high': 'lhigh', 'open': 'lopen'}, inplace=True)
        if 'nclose' in dm.columns:
            dd = cct.combine_dataFrame(
                df, dm.loc
                [:,
                 ['nclose', 'close', 'name', 'open', 'high', 'low', 'llastp',
                  'ticktime']],
                clean=True)
        else:
            dd = cct.combine_dataFrame(
                df, dm.loc
                [:, ['close', 'name', 'open', 'high', 'low', 'llastp', 'ticktime']],
                clean=True)
        dd['perc'] = ((dd['close'] - dd['llastp']) / dd['llastp'] *
                      100).map(lambda x: round(x, 2))
        # dd = cct.combine_dataFrame(df, dm['name'])

    else:
        dd = cct.combine_dataFrame(df, dm['name'])
    # cname = sina_data.Sina().get_code_cname(code)
    return dd

from JSONData import tdx_hdf5_api as h5a
def get_tdx_all_from_h5(resample='d',showtable=True):
    #sina_monitor
    h5_fname = 'tdx_last_df'
    resample=resample
    if resample=='d':
        dl='60'
    else:
        dl='200'
    filter='y'
    h5_table = 'low' + '_' + resample + '_' + str(dl) + '_' + filter + '_' + 'all'
    h5 = h5a.load_hdf_db(h5_fname, table=h5_table,code_l=None, timelimit=False,showtable=showtable)
    return h5


def get_tdx_all_from_h5(resample='d',dl=ct.duration_date_day, end=None, ptype='low', filter='y',showtable=True):
    if end is not None:
        h5_table = ptype + '_' + resample + '_' + str(dl) + '_' + filter + \
            '_' + end.replace('-', '') + '_' + 'all'
    else:
        h5_table = ptype + '_' + resample + '_' + str(dl) + '_' + filter + '_' + 'all'
    #sina_monitor
    h5_fname = 'tdx_last_df'
    resample=resample
    if resample=='d' or resample=='3d':
        dl='60'
    else:
        dl='200'
    filter='y'
    h5_table = 'low' + '_' + resample + '_' + str(dl) + '_' + filter + '_' + 'all'
    h5 = h5a.load_hdf_db(h5_fname, table=h5_table,code_l=None, timelimit=False,showtable=showtable)
    return h5

## init data

### get_tdx_all

In [42]:
h5_day=get_tdx_all_from_h5(resample='d')

keys:['/low_3d_60_y_all', '/low_d_60_y_all', '/low_m_300_y_all', '/low_w_200_y_all']


### sina

In [27]:
sina_dm = sina_data.Sina().market('all')
lastday = cct.last_tddate()
lastday,sina_dm.name[:1]
sina_dm = clean_sina_dm(sina_dm)

('2025-06-16',
 code
 300426    华智数媒
 Name: name, dtype: object)

In [28]:
cct.get_time_to_date(time.time(), format='%Y-%D %H:%M:%S')

# abc=dm.loc[:,'ticktime'][-1]
sina_dm.ticktime[-2:]
# dm.ticktime[:1]
# sina_dm.loc[:, 'ticktime'][-1]
# time.strptime(abc,format='%Y-%m-%d %H:%M:%S')
# (time.mktime(abc.timetuple()))
(time.time() - time.mktime(sina_dm.loc[:, 'ticktime'][-1].timetuple())) / 60

'2025-06/17/25 10:24:37'

code
688147   2025-06-17 10:24:00
600185   2025-06-17 10:24:05
Name: ticktime, dtype: datetime64[ns]

0.539786167939504

#### 实时成交

In [30]:
def apply_col_toint(df, col=None):
    if col is None:
        co2int = ['boll', 'op', 'ratio', 'fib', 'fibl', 'df2']
    # co2int.extend([co for co in df.columns.tolist()
    #                if co.startswith('perc') and co.endswith('d')])
        co2int.extend(['top10', 'topR'])
    else:
        co2int = col
    co2int = [inx for inx in co2int if inx in df.columns]

    for co in co2int:
        df[co] = df[co].astype(int)

    return df

def readHdf5(fpath, root=None):
    store = pd.HDFStore(fpath, "r")
    print(store.keys())
    if root is None and store is not None:
        root = store.keys()[0].replace("/", "")
    df = store[root]
    df = apply_col_toint(df)
    store.close()
    return df

In [31]:
# tdx_path = "G:\\tdx_last_df.h5"
# from JohnsonUtil import commonTips as cct
sina_MultiD_path = f"{cct.get_ramdisk_dir()}\\sina_MultiIndex_data.h5".replace('\\',os.sep)
print(f'sina_MultiD_path:{sina_MultiD_path}')
h5 = readHdf5(sina_MultiD_path)
print(sina_MultiD_path)
h5.shape
h5code='603019'
h5.loc[h5code][-3:]
code_muti = '603019'
# h5.loc[code_muti][:2]

freq = 'D'
# freq = '5M'
startime = '09:25:00'
endtime = '15:01:00'

time_ratio = cct.get_work_time_ratio()
time_ratio
run_col = ['close', 'volume']
# run_col = ['close']
mdf = cct.get_limit_multiIndex_freq(
    h5, freq=freq.upper(),
    col=run_col, start=startime, end=endtime, code=None)
allvol = mdf.volume.sum()
allvol = round(allvol / 10 / 1000 / 1000,2)
print("总量:%s 亿"%(allvol))
mdf['vol']=mdf['volume'].apply(lambda x: round(x /1000/1000,2)) 
mdf['amount']=(mdf.volume * mdf.close).map(lambda x: round(x/100/1000/1000, 2))
#全局占比
mdf['volume'] = mdf.amount.apply(
    lambda x: round(x / allvol * 100 / time_ratio, 2))
mdf=mdf.reset_index().set_index('code')
mdf.loc[h5code]
str(h5.index.get_level_values(1)[-1])[:10]

sina_MultiD_path:/Volumes/RamDisk/sina_MultiIndex_data.h5
['/all_30']
/Volumes/RamDisk/sina_MultiIndex_data.h5


(700855, 6)

,close,high,low,llastp,volume,lastbuy
ticktime,,,,,,
2025-06-17 10:23:24,67.84,68.79,67.81,68.82,11235340,68.5
2025-06-17 10:23:45,67.87,68.79,67.81,68.82,11287540,68.5
2025-06-17 10:24:06,67.89,68.79,67.81,68.82,11324140,68.5


0.508

总量:4250.19 亿


ticktime    2025-06-18 00:00:00
close                 68.134667
volume                     0.36
vol                       11.32
amount                     7.72
Name: 603019, dtype: object

'2025-06-17'

#### 读取实时成交个股

In [32]:
nowdf = mdf.sort_values(by=['volume'], ascending=[0])
# nowdf.shape, df.shape, code_idx.shape
# top20_nd = nowdf.reset_index().loc[:, ['code', 'volume']][:20]
top20_nd = nowdf.reset_index().loc[:, ['code', 'volume']][:20]
top20_nd = get_sina_data_cname(top20_nd)
nowdf.shape, top20_nd

((5410, 5),
         volume  name
 code                
 300468    1.96  四方精创
 300251    1.75  光线传媒
 300773    1.65   拉卡拉
 002104    1.20  恒宝股份
 002195    1.19  岩山科技
 600570    1.19  恒生电子
 300872    1.07  天阳科技
 002475    1.01  立讯精密
 600366    1.01  宁波韵升
 300682    0.92  朗新集团
 300308    0.91  中际旭创
 300502    0.88   新易盛
 002366    0.80  融发核电
 688256    0.80   寒武纪
 002241    0.78  歌尔股份
 300199    0.78  翰宇药业
 002384    0.77  东山精密
 300476    0.76  胜宏科技
 002537    0.75  海联金汇
 600519    0.73  贵州茅台)

##### 合并实时成交量及SinaDM

In [33]:
sina_dm.shape

(5180, 41)

In [34]:
# mdf[:1]
sinadm = cct.combine_dataFrame(sina_dm, mdf[['vol','amount']])
sinadm['percent']=((sinadm['close']-sinadm['llastp'])/(sinadm['llastp'])*100).round(1)

## resample 3D

In [43]:
h5_3d = get_tdx_all_from_h5(resample='3d')
tdx_3d = cct.combine_dataFrame(h5_3d,sinadm)

keys:['/low_3d_60_y_all', '/low_d_60_y_all', '/low_m_300_y_all', '/low_w_200_y_all']


In [44]:
tdx_3d.shape
# tdx_now.query('close >  lastp1d  and lastl1d < ma51d and close > ma51d and lastp1d < upper and lastp1d > lastp2d and lasth1d >high4 and boll > 0 and volume < 1')
tdx_select = tdx_3d.query('close > lastp1d and percent < 3')[:10]
show_tdx_data(tdx_select)

(5177, 217)

Count:10 217


,name,date,close,hmax,lmin,perc1d,nclose,llow,perc3d,perc5d
code,,,,,,,,,,
000001,平安银行,2025-04-07,11.8,11.5,10.1,0.1,11.8,10.1,0.0,1.3
000002,万 科Ａ,2025-06-13,6.5,7.3,6.5,-0.1,6.5,6.4,0.1,-0.1
000004,*ST国华,2025-05-07,9.5,10.4,7.2,-1.1,9.5,7.2,0.0,1.2
000020,深华发Ａ,2025-04-10,12.5,12.6,9.2,-0.1,12.5,9.2,1.3,0.0
000021,深科技,2025-04-07,17.2,19.3,14.7,-0.1,17.2,16.5,0.1,0.0
000032,深桑达Ａ,2025-06-13,19.2,23.2,18.0,-1.1,19.2,19.0,0.1,0.2
000034,神州数码,2025-04-10,36.2,42.8,31.3,-1.1,36.3,31.3,1.2,0.0
000042,中洲控股,2025-04-07,7.0,7.2,5.9,0.1,7.0,5.9,0.0,0.0
000048,京基智农,2025-04-18,16.1,18.0,14.5,-0.1,16.0,14.9,0.1,0.1


## resample W

In [45]:
h5_w = get_tdx_all_from_h5(resample='w')
tdx_w = cct.combine_dataFrame(h5_w,sinadm)

keys:['/low_3d_60_y_all', '/low_d_60_y_all', '/low_m_300_y_all', '/low_w_200_y_all']


In [46]:
tdx_w.shape
# tdx_now.query('close >  lastp1d  and lastl1d < ma51d and close > ma51d and lastp1d < upper and lastp1d > lastp2d and lasth1d >high4 and boll > 0 and volume < 1')
tdx_select_w = tdx_w.query('lasth1d > lasth2d and lasth2d >lasth3d and lasth2d > upper1')[:10]
show_tdx_data(tdx_select_w)

(5180, 219)

Count:10 219


,name,date,close,hmax,lmin,perc1d,nclose,llow,perc3d,perc5d
code,,,,,,,,,,
000029,深深房Ａ,2025-01-10,19.8,18.0,12.8,1.2,20.0,13.9,23.8,1.3
000513,丽珠集团,2025-03-07,36.0,38.2,32.9,1.2,36.4,34.1,0.1,1.2
000533,顺钠股份,2024-11-22,7.2,7.4,3.9,1.2,7.2,3.9,1.2,1.2
000599,青岛双星,2025-01-10,5.0,5.3,4.1,1.2,5.0,4.1,0.1,1.3
000701,厦门信达,2025-01-03,7.7,5.7,3.7,50.6,7.7,4.3,10.1,0.0
000766,通化金马,2024-11-22,27.1,20.0,13.6,46.5,27.2,13.7,1.3,20.5
000831,中国稀土,2025-01-03,36.9,34.2,26.9,1.2,36.8,26.9,1.1,1.3
000919,金陵药业,2025-01-10,6.9,7.1,5.9,1.2,6.9,6.3,0.0,0.1
000963,华东医药,2025-01-10,42.7,42.8,30.4,1.2,43.4,30.4,28.4,8.9
